<a href="https://colab.research.google.com/github/edyoda/AI-Agent-Development-and-GenAI/blob/main/Day2_GenAI_Foundations_GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

To understand the foundations of Generative AI (GenAI), it's important to start with the basics of machine learning, deep learning, and neural networks. Below is a Python code example that demonstrates the foundational concepts of Generative AI using a simple Generative Adversarial Network (GAN). This example will help you understand how generative models work.

**Step 1: Import Libraries**

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt

**Step 2: Define the Generator and Discriminator**

The generator creates fake data, and the discriminator tries to distinguish between real and fake data.

In [ ]:
# Generator model
def build_generator(latent_dim):
    model = tf.keras.Sequential([
        layers.Dense(128, input_dim=latent_dim, activation='relu'),
        layers.Dense(256, activation='relu'),
        layers.Dense(512, activation='relu'),
        layers.Dense(784, activation='sigmoid'),  # 28x28 for MNIST images
        layers.Reshape((28, 28, 1))
    ])
    return model

# Discriminator model
def build_discriminator(img_shape):
    model = tf.keras.Sequential([
        layers.Flatten(input_shape=img_shape),
        layers.Dense(512, activation='relu'),
        layers.Dense(256, activation='relu'),
        layers.Dense(128, activation='relu'),
        layers.Dense(1, activation='sigmoid')  # Binary classification (real/fake)
    ])
    return model

**Step 3: Define the GAN**

Combine the generator and discriminator into a GAN model.

In [ ]:
def build_gan(generator, discriminator):
    discriminator.trainable = False  # Freeze discriminator during generator training
    model = tf.keras.Sequential([
        generator,
        discriminator
    ])
    return model

**Step 4: Load and Preprocess Data**

For this example, we'll use the MNIST dataset.

In [ ]:
# Load MNIST dataset
(X_train, _), (_, _) = tf.keras.datasets.mnist.load_data()
X_train = X_train / 255.0  # Normalize to [0, 1]
X_train = np.expand_dims(X_train, axis=-1)  # Add channel dimension

**Step 5: Compile Models**

Compile the discriminator and GAN.


In [ ]:
# Hyperparameters
latent_dim = 100
img_shape = (28, 28, 1)

# Build and compile discriminator
discriminator = build_discriminator(img_shape)
discriminator.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Build generator
generator = build_generator(latent_dim)

# Build and compile GAN
gan = build_gan(generator, discriminator)
gan.compile(optimizer='adam', loss='binary_crossentropy')

**Step 6: Train the GAN**

Train the GAN in alternating steps: train the discriminator on real and fake data, then train the generator to fool the discriminator.

In [ ]:
# Training loop
epochs = 10
batch_size = 64

for epoch in range(epochs):
    # Train discriminator
    idx = np.random.randint(0, X_train.shape[0], batch_size)
    real_imgs = X_train[idx]

    noise = np.random.normal(0, 1, (batch_size, latent_dim))
    fake_imgs = generator.predict(noise)

    # Labels for real and fake images
    real_labels = np.ones((batch_size, 1))
    fake_labels = np.zeros((batch_size, 1))

    # Train discriminator on real and fake images
    d_loss_real = discriminator.train_on_batch(real_imgs, real_labels)
    d_loss_fake = discriminator.train_on_batch(fake_imgs, fake_labels)
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

    # Train generator
    noise = np.random.normal(0, 1, (batch_size, latent_dim))
    g_loss = gan.train_on_batch(noise, real_labels)  # Generator tries to fool discriminator

    # Print progress

    print(f"Epoch: {epoch}, D Loss: {d_loss[0]}, G Loss: {g_loss}")
    # Generate and save sample images
    sample_images(generator, epoch)

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
Epoch: 0, D Loss: 1.6128430366516113, G Loss: 0.20828667283058167
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Epoch: 1, D Loss: 1.6502070426940918, G Loss: 0.19951413571834564
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
Epoch: 2, D Loss: 1.686079978942871, G Loss: 0.19146983325481415
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Epoch: 3, D Loss: 1.7197588682174683, G Loss: 0.1840730458498001
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
Epoch: 4, D Loss: 1.7511080503463745, G Loss: 0.17724888026714325
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
Epoch: 5, D Loss: 1.7808496952056885, G Loss: 0.17093497514724731
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
Epoch: 6, D Loss: 1.8089277744293213, G Loss: 0.16507670283317566
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
2/2 ━━━━

**Step 7: Generate Sample Images**

Define a function to generate and save sample images during training.

In [ ]:
def sample_images(generator, epoch, n=5):
    noise = np.random.normal(0, 1, (n * n, latent_dim))
    gen_imgs = generator.predict(noise)

    # Rescale images to [0, 1]
    gen_imgs = 0.5 * gen_imgs + 0.5

    fig, axs = plt.subplots(n, n)
    cnt = 0
    for i in range(n):
        for j in range(n):
            axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
            axs[i,j].axis('off')
            cnt += 1
    fig.savefig(f"gan_images/epoch_{epoch}.png")
    plt.close()

**Key Concepts in This Code:**

Generator: Creates fake data (e.g., images).

Discriminator: Distinguishes between real and fake data.

Adversarial Training: The generator and discriminator are trained simultaneously in a competitive manner.

Latent Space: The generator uses random noise (latent vectors) to create data.

Loss Functions: Binary cross-entropy is used to measure how well the discriminator and generator perform.

This is a basic implementation of a GAN. Modern Generative AI models like GPT, DALL·E, and Stable Diffusion build on these foundational concepts but use more advanced architectures and techniques.